In [ ]:
import definicoes *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

path_pcappi_dbz = './Pyradar_pseudo_cappi_spol_2km/'
dados_pcappi_dbz = glob.glob(path_pcappi_dbz+'*PsCAPPI.npy')
dados_pcappi_chuva = glob.glob(path_pcappi_dbz+'*chuva_mmh.npy')

valores_dbz = []
import pandas as pd

for arq_chuva,arq_dbz in tqdm(zip(dados_pcappi_chuva,dados_pcappi_dbz),total = len(dados_pcappi_chuva)):
  datahora = dt_from_str(os.path.basename(arq_chuva)[:14])
  chuva = np.load(arq_chuva)
  dbz = np.load(arq_dbz)
  valores_dbz.append([datahora,chuva.sum(), np.nanmean(dbz)])
x = pd.DataFrame(valores_dbz,columns=['Data','Chuva','dbz'])

# Assuming 'x' DataFrame is already defined as in your provided code
plt.rcParams["font.family"] = "Liberation Serif"
fig, ax1 = plt.subplots(figsize=(10, 5))

# Highlight the periods of interest
start1, end1 = "16:40", "17:40"
start2, end2 = "18:30", "19:30"

ax1.axvspan(
    pd.Timestamp(f'2019-01-08 {start1}'),
    pd.Timestamp(f'2019-01-08 {end1}'),
    color='tab:pink',
    alpha=0.5,
    label='Período 1'
)
ax1.axvspan(
    pd.Timestamp(f'2019-01-08 {start2}'),
    pd.Timestamp(f'2019-01-08 {end2}'),
    color='tab:green',
    alpha=0.5,
    label='Período 2'
)

# Plot Chuva on the first y-axis
color = 'tab:blue'
ax1.set_xlabel('Hora (UTC)')
ax1.set_ylabel('Chuva (mm/área)', color=color,fontweight='bold')
ax1.plot(x['Data'][97:], x['Chuva'][97:], color=color, label='Chuva',linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(axis='x')

# Create a second y-axis sharing the same x-axis
ax2 = ax1.twinx()

# Plot dBZ on the second y-axis
color = 'tab:orange'
ax2.set_ylabel('Refletividade (dBZ/área)', color=color,fontweight='bold')
ax2.plot(x['Data'][97:], x['dbz'][97:], color=color, label='Refletividade',linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

ax2.set_xlim(pd.Timestamp(f'2019-01-08 12:00'),pd.Timestamp(f'2019-01-08 20:40'))
ax1.set_ylim(0)

# Combine legends from both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.xticks(rotation=45)
fig.autofmt_xdate()
plt.suptitle(' Evolução da Refletividade e Precipitação Acumulada Estimada na Área de Estudo',fontweight='bold')
plt.title(' Data: 08/01/2019'+' '*(168)+'Radar: SPOL',loc='left',fontsize=10)
plt.savefig('./figura.png',dpi=1200)
plt.show()

In [ ]:
#@title Rodada Rainymotion
# data_hora = '201901081800'; periodo = 2
data_hora = '201901081640'; periodo = 1
lead_steps = 12



from rainymotion.models import  Persistence,SparseSD, Sparse ,Dense, DenseRotation

model = SparseSD()
model.input_data = localiza_data(dados_pcappi_dbz,data_hora,2) #utiliza duas imagens de radar mais recentes
model.lead_steps = lead_steps

# Testando todas as combinações de warper e extrapolation
extrapolation_ = "simple_delta"#, "linear"]
warper_ = "euclidean"#"projective"#"similarity"#"affine"#

model.warper = warper_
model.extrapolation = extrapolation_

nowcast_SparseSD = model.run()

from rainymotion.models import Sparse

# Criar modelo
model = Sparse()
model.input_data = localiza_data(dados_pcappi_dbz,data_hora,18)
model.lead_steps = lead_steps

# Configurações possíveis
extrapolation_ = "simple_delta"#"linear"#
warper_ = "euclidean"#"projective"#"similarity"#"affine"#

model.warper = warper_
model.extrapolation = extrapolation_

nowcast_Sparse = model.run()

from rainymotion.models import Persistence

# Criar o modelo
model = Persistence()
model.input_data = localiza_data(dados_pcappi_dbz,data_hora,20)
model.lead_steps = lead_steps

nowcast_Persistence = model.run()
print("O formato da matriz SparseSD é:\t\t",nowcast_SparseSD.shape)
print("O formato da matriz Sparse é:\t\t",nowcast_Sparse.shape)
print("O formato da matriz Persistence é:\t",nowcast_Persistence.shape)

In [ ]:
import pysteps
from pysteps.datasets import load_dataset

precipitation, lista_name_arq = localiza_data(dados_pcappi_chuva,data_hora,20,pysteps=True)
precipitation.shape
metadata = Create_metadata(unit='mm/h',lista_name_arq=lista_name_arq)

train_precip = precipitation[:]
observed_precip = localiza_data(dados_pcappi_chuva,data_hora,lead_steps,sentido='frente')

from pysteps import motion
from pysteps.visualization import plot_precip_field, quiver
from pysteps.utils import transformation

# Importar o algoritmo de fluxo óptico de Lucas-Kanade
oflow_method = motion.get_method("LK")

train_precip_dbr, metadata_dbr = transformation.dB_transform(
    train_precip, metadata, threshold=0.1, zerovalue=-15)

motion_field = oflow_method(train_precip_dbr)

from pysteps import nowcasts

# Extrapolar a última observação do radar
extrapolate = nowcasts.get_method("extrapolation")

n_leadtimes = observed_precip.shape[0]

precip_forecast = extrapolate(train_precip[-1], motion_field, n_leadtimes)
precip_forecast[np.isnan(precip_forecast)] = 0
print("The shape of the resulting array is: ", precip_forecast.shape)


In [ ]:
#@title Calculando Metricas
# Observados
observado_dbz = localiza_data(dados_pcappi_dbz,data_hora,lead_steps,sentido='frente')
observado_chuva,name_obs = localiza_data(dados_pcappi_chuva,data_hora,lead_steps,sentido='frente',pysteps=True)

# Rainymotion X Observado (dbz)
df_obs_SparseSD = Calc_metricas(observado_dbz,nowcast_SparseSD)
df_obs_Sparse = Calc_metricas(observado_dbz,nowcast_Sparse)
df_obs_Persistence = Calc_metricas(observado_dbz,nowcast_Persistence)

# Pysteps (dbz)
pysteps_dbz = np.array([ZR_Marshall(chuva,sentido='chuva_to_dbz') for chuva in precip_forecast])
# Pysteps X Observado (dbz)
df_obs_precip_forecast_dbz = Calc_metricas(observado_dbz,pysteps_dbz)

# Rainymotion X Pysteps (dbz)
df_SparseSD_precip_forecast = Calc_metricas(nowcast_SparseSD,pysteps_dbz)
df_Sparse_precip_forecast = Calc_metricas(nowcast_Sparse,pysteps_dbz)
df_Persistence_precip_forecast = Calc_metricas(nowcast_Persistence,pysteps_dbz)

# Pysteps X Observado (Chuva)
df_obs_precip_forecast = Calc_metricas(observado_chuva,precip_forecast)

# Rainymotion (Chuva)
nowcast_SparseSD_chuva = ZR_Marshall(nowcast_SparseSD,sentido='dbz_to_chuva')
nowcast_Sparse_chuva = ZR_Marshall(nowcast_Sparse,sentido='dbz_to_chuva')
nowcast_Persistence_chuva = ZR_Marshall(nowcast_Persistence,sentido='dbz_to_chuva')

# Rainymotion X Observado (Chuva)
df_obs_SparseSD_chuva = Calc_metricas(observado_chuva,nowcast_SparseSD_chuva)
df_obs_Sparse_chuva = Calc_metricas(observado_chuva,nowcast_Sparse_chuva)
df_obs_Persistence_chuva = Calc_metricas(observado_chuva,nowcast_Persistence_chuva)

# Rainymotion X Pysteps (Chuva)
df_SparseSD_precip_forecast_chuva = Calc_metricas(nowcast_SparseSD_chuva,precip_forecast)
df_Sparse_precip_forecast_chuva = Calc_metricas(nowcast_Sparse_chuva,precip_forecast)
df_Persistence_precip_forecast_chuva = Calc_metricas(nowcast_Persistence_chuva,precip_forecast)


In [ ]:
#@title FigurasTaxa de Precipitação
import matplotlib.pyplot as plt


metricas_legendas = {
    "CSI": "Índice de Sucesso Crítico (CSI)",
    "FAR": "Taxa de Alarmes Falsos (FAR)",
    "POD": "Probabilidade de Detecção (POD)",
    "MAE": "Erro Absoluto Médio (MAE)",
    "HSS": "Índice de Habilidade (HSS)",
    "TSS": "Índice de True Skill (TSS)",
    "R": "Coeficiente de Correlação de Pearson (R)"
}

# Criando uma figura com 2 linhas e 3 colunas de subgráficos
fig, axs = plt.subplots(2, 3, figsize=(12, 8))

# Ajustando o espaçamento entre os gráficos
plt.tight_layout()

for idx, indice in enumerate(df_obs_SparseSD.columns[1:-1]):
    row = idx // 3
    col = idx % 3

    ax = axs[row, col]
    eixox = df_obs_SparseSD['step'] * 5
    ax.plot(eixox, df_obs_SparseSD_chuva[indice], label='SparseSD')
    ax.plot(eixox, df_obs_Sparse_chuva[indice], label='Sparse')
    ax.plot(eixox, df_obs_Persistence_chuva[indice], label='Persistence')
    ax.plot(eixox, df_obs_precip_forecast[indice], label='Pysteps')
    # display(df_obs_precip_forecast_dbz[indice])
    if indice == 'mae':
      ax.set_ylabel('mm/h')
    ax.grid(visible=True, linestyle="--", alpha=0.7)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_title(indice.upper(), fontsize=20)
    ax.set_xlabel('Passos de Tempo (min)', fontsize=16)

# Adicionando uma única legenda na parte inferior
handles, labels = ax.get_legend_handles_labels()
plt.suptitle(f'Taxa de Precipitação\nComparação entre Observado e o Nowcast (Período {periodo})',
             fontweight = 'bold',fontsize=24,y=1.01)
fig.legend(handles, labels, loc='lower center', ncol=4,bbox_to_anchor=(0.5, -0.05), fontsize=20, frameon=False)  # Ajustar 'ncol' conforme necessário
fig.tight_layout()
plt.show()
fig = plt.figure(figsize=(5,4))
for idx, indice in enumerate(df_obs_SparseSD.columns[-1:]):
    if indice == 'r':
        plt.plot(eixox, df_SparseSD_precip_forecast_chuva[indice], label=f'SparseSD X Pysteps')
        plt.plot(eixox, df_Sparse_precip_forecast_chuva[indice], label=f'Sparse X Pysteps')
        plt.plot(eixox, df_Persistence_precip_forecast_chuva[indice], label=f'Persistence X Pysteps')
    plt.xlabel('Passos de Tempo (min)')
    plt.title('Taxa de Precipitação'+'\n'+metricas_legendas[indice.upper()], fontsize=14)
    plt.grid(visible=True, linestyle="--", alpha=0.7)
    plt.legend(fontsize=12)
plt.show()

In [ ]:
#@title Figuras Refletividade
import matplotlib.pyplot as plt

metricas_legendas = {
    "CSI": "Índice de Sucesso Crítico (CSI)",
    "FAR": "Taxa de Alarmes Falsos (FAR)",
    "POD": "Probabilidade de Detecção (POD)",
    "MAE": "Erro Absoluto Médio (MAE)",
    "HSS": "Índice de Habilidade (HSS)",
    "TSS": "Índice de True Skill (TSS)",
    "R": "Coeficiente de Correlação de Pearson (R)"
}

# Criando uma figura com 2 linhas e 3 colunas de subgráficos
fig, axs = plt.subplots(2, 3, figsize=(12, 8))

# Ajustando o espaçamento entre os gráficos
plt.tight_layout()

for idx, indice in enumerate(df_obs_SparseSD.columns[1:-1]):
    row = idx // 3
    col = idx % 3

    ax = axs[row, col]
    eixox = df_obs_SparseSD['step'] * 5
    ax.plot(eixox, df_obs_SparseSD[indice], label='SparseSD')
    ax.plot(eixox, df_obs_Sparse[indice], label='Sparse')
    ax.plot(eixox, df_obs_Persistence[indice], label='Persistence')
    ax.plot(eixox, df_obs_precip_forecast_dbz[indice], label='Pysteps')
    # display(df_obs_precip_forecast_dbz[indice])
    if indice == 'mae':
      ax.set_ylabel('dBZ')
    ax.grid(visible=True, linestyle="--", alpha=0.7)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_title(indice.upper(), fontsize=20)
    ax.set_xlabel('Passos de tempo (minutos)', fontsize=16)

# Adicionando uma única legenda na parte inferior
handles, labels = ax.get_legend_handles_labels()
plt.suptitle(f'Refletividade\nComparação entre Observado e o Nowcast (Período {periodo})',
             fontweight = 'bold',fontsize=24,y=1.01)
fig.legend(handles, labels, loc='lower center', ncol=4,bbox_to_anchor=(0.5, -0.05), fontsize=20, frameon=False)  # Ajustar 'ncol' conforme necessário
fig.tight_layout()
plt.show()
fig = plt.figure(figsize=(5,4))
for idx, indice in enumerate(df_obs_SparseSD.columns[-1:]):
    if indice == 'r':
        plt.plot(eixox, df_SparseSD_precip_forecast[indice], label=f'SparseSD X Pysteps')
        plt.plot(eixox, df_Sparse_precip_forecast[indice], label=f'Sparse X Pysteps')
        plt.plot(eixox, df_Persistence_precip_forecast[indice], label=f'Persistence X Pysteps')
    plt.xlabel('Passos de tempo (min)')
    plt.title('Refletividade'+'\n'+metricas_legendas[indice.upper()], fontsize=14)
    plt.grid(visible=True, linestyle="--", alpha=0.7)
    plt.legend(fontsize=12)
plt.show()

In [ ]:
#@title Mapas de celulas

path_dados_dBZ = './dBZ/'
fglob_dbz =sorted(glob.glob(path_dados_dBZ+'*.vol'))

rbdict = wrl.io.read_rainbow(fglob_dbz[128], loaddata=True)
s = data_input(rbdict)
refl= s[0]
azi= s[8]
r= s[9]
latitude= s[4]
longitude= s[5]
altitude= s[6]
datamax= s[3]
datamin= s[2]
site_coords = (longitude, latitude, altitude)
data = refl
# data[data<0]=0
azimuth =azi
ranges =r
#########################################

#########################################
d_grid = 2 # km
nowcastings= []

for now in precip_forecast:
  m_base = now
  m_base[m_base<=0] = -9999
  m_base = np.expand_dims(m_base, axis=0)

  m_base = transform_grid(m_base, time_data=np.arange(1, 365 * 0.2, 0.2),
                    x_data=np.arange(-120000, 120000, d_grid*1000),y_data=np.arange(-120000, 120000, d_grid*1000),z_data=np.arange(0, 1),
                    latitude=latitude, longitude=longitude, altitude=altitude, datamax=datamax, datamin=datamin,
                        )
  nowcastings.append(m_base)
plot_grid_reflectivity(nowcastings[::2],title=[datetime.datetime.strptime(nome[:14], "%Y%m%d%H%M%S").strftime("%d/%m/%Y %H:%M:%S")+f' (passo {i})' for i,nome in enumerate(name_obs,start=0)][1::2],
                       lat_lines= np.arange(-24.8, -22.2, 0.2)[::2], lon_lines = np.arange(-47.5, -44, 1),
                       opt_title_cbar={'label':'Taxa de Precipitação (mm/h)',"fontsize": 16},
                       opt_title_fig={'t':f'Nowcast Estimado pelo Modelo Pysteps (Período {periodo})',
                                      'fontweight':'bold',
                                      'y':0.64,"fontsize": 22})

#########################################


#########################################



nowcastings= []
for now in nowcast_Persistence:
  m_base = now
  m_base = ZR_Marshall(m_base,sentido='dbz_to_chuva')
  m_base[m_base<=0.05] = np.nan
  m_base = np.expand_dims(m_base, axis=0)

  m_base = transform_grid(m_base, time_data=np.arange(1, 365 * 0.2, 0.2),
                    x_data=np.arange(-120000, 120000, d_grid*1000),y_data=np.arange(-120000, 120000, d_grid*1000),z_data=np.arange(0, 1),
                    latitude=latitude, longitude=longitude, altitude=altitude, datamax=datamax, datamin=datamin,
                        )
  nowcastings.append(m_base)
plot_grid_reflectivity(nowcastings[::2],title=[datetime.datetime.strptime(nome[:14], "%Y%m%d%H%M%S").strftime("%d/%m/%Y %H:%M:%S")+f' (passo {i})' for i,nome in enumerate(name_obs,start=0)][1::2],
                       lat_lines= np.arange(-24.8, -22.2, 0.2)[::2], lon_lines = np.arange(-47.5, -44, 1),
                       opt_title_cbar={'label':'Taxa de Precipitação (mm/h)',"fontsize": 16},
                       opt_title_fig={'t':f'Nowcast Estimado pelo Modelo Rainymotion - Persistence (Período {periodo})',
                                      'fontweight':'bold',
                                      'y':0.64,"fontsize": 22})


observado= []
for now in observado_chuva:
  m_base = now
  m_base[m_base<=0] = -9999
  m_base = np.expand_dims(m_base, axis=0)

  m_base = transform_grid(m_base, time_data=np.arange(1, 365 * 0.2, 0.2),
                    x_data=np.arange(-120000, 120000, d_grid*1000),y_data=np.arange(-120000, 120000, d_grid*1000),z_data=np.arange(0, 1),
                    latitude=latitude, longitude=longitude, altitude=altitude, datamax=datamax, datamin=datamin,
                        )
  observado.append(m_base)
plot_grid_reflectivity(observado[1::2],title=[datetime.datetime.strptime(nome[:14], "%Y%m%d%H%M%S").strftime("%d/%m/%Y %H:%M:%S") for nome in name_obs][1::2],
                       lat_lines= np.arange(-24.8, -22.2, 0.2)[::2], lon_lines = np.arange(-47.5, -44, 1),
                       opt_title_cbar={'label':'Taxa de Precipitação (mm/h)',"fontsize": 16},
                       opt_title_fig={'t':f'Precipitação Estimada na Área de Estudo (Período {periodo})',
                                      'fontweight':'bold',
                                      'y':0.64,"fontsize": 22})

